In [1]:
import numpy as np
import pandas as pd
import loadData
import monteCarloClass
import time

In [2]:
data, dataName = loadData.loadData("mt_all.csv")
intensityVariable = "deltaDurS"
sensoryVar = "audNoise"
standardVar = "standardDur"
conflictVar = "conflictDur"
#s
visualStandardVar = "unbiasedVisualStandardDur"
visualTestVar = "unbiasedVisualTestDur"
audioStandardVar = "standardDur"
audioTestVar = "testDurS"
# Instantiate the Monte Carlo class
mc_fitter = monteCarloClass.OmerMonteCarlo(data)
print("Visual test duration range:", data["recordedDurVisualTest"].min(), "to", data["recordedDurVisualTest"].max())
# fit parameters
mc_fitter.nStart = 1 # Number of random starts for optimization
mc_fitter.nSimul = 500
mc_fitter.optimizationMethod= "bads"  # Use BADS for optimization
mc_fitter.modelName = "lognorm"  # Set measurement distribution to Gaussian
mc_fitter.integrationMethod = "analytical"  # Use numerical integration instead of analytical
mc_fitter.freeP_c = False  # Allow different prior widths for conflict conditions
mc_fitter.sharedLambda = False  # Use separate lapse rates for each condition
mc_fitter.dataName = dataName



 Total trials before cleaning
: 2156
uniqueSensory: [1.2 0.1] 
 uniqueStandard: [0.5] 
 uniqueConflict: [np.float64(-0.25), np.float64(-0.17), np.float64(-0.08), np.float64(0.0), np.float64(0.08), np.float64(0.17), np.float64(0.25)]
total trials after cleaning: 2156
Visual test duration range: 0.1072 to 1.0763


In [3]:
# read back the saved file to verify
import os
import json
import loadResults
try:
    res=loadResults.loadFitResults(mc_fitter,dataName, modelName=mc_fitter.modelName)
    res_with_new_lines = json.dumps(res, indent=4)
    print("Loaded saved fit results:\n")
    print(res_with_new_lines)
    #print(f"Loaded saved fit results: {res}")
    mc_fitter.modelFit= res['fittedParams']
    mc_fitter.logLikelihood= res['logLikelihood']
    mc_fitter.aic= res['AIC']
    mc_fitter.bic= res['BIC']
    mc_fitter.nDataPoints= res['n_conditions']
except:
    print("No saved fit results found, proceeding to fit the model.")
    user_input = input("Do you want to proceed with model fitting? (yes/no): ").strip().lower()
    if user_input != 'yes':
        print("Model fitting skipped.")
        
    timeStart = time.time()
    print(f"\nFitting Causal Inference Model for {dataName} with {len(mc_fitter.groupedData)} unique conditions")
    fittedParams = mc_fitter.fitCausalInferenceMonteCarlo(mc_fitter.groupedData)
    print(f"\nFitted parameters for {dataName}: {fittedParams}")
    print(f"Time taken to fit: {time.time() - timeStart:.2f} seconds")
    mc_fitter.modelFit= fittedParams
    mc_fitter.logLikelihood= -mc_fitter.nLLMonteCarloCausal(fittedParams, mc_fitter.groupedData)    
    
    #save the fit results
    import fitSaver
    fitSaver.saveFitResultsSingle(mc_fitter, mc_fitter.modelFit, dataName)

Looking for saved fit: mt_all with model lognorm_LapseFree_sharedPrior
Loaded saved fit results:

{
    "participantID": "mt",
    "modelType": "lognorm_LapseFree_sharedPrior",
    "fittedParams": [
        0.032721803495229546,
        0.15576154849977908,
        0.3718890945007365,
        0.7982787610074273,
        0.46007481075666873,
        0.14598556138063035,
        0.21001085452070584,
        0.5957054038783781,
        2.1428683373655852
    ],
    "AIC": 2209.9903735016615,
    "BIC": 2244.9151703792677,
    "logLikelihood": -1095.9951867508307,
    "n_conditions": 358
}


## Simulate data using fitted parameters

In [4]:
#proceed to simulate and save to {filename}
import fitSaver
import os
def loadSimulatedData(mc_fitter, dataName,participantID, modelType=mc_fitter.modelName):
    participantID = dataName.split(".csv")[0]
    #modelType = mc_fitter.modelName

    if mc_fitter.sharedLambda:
        modelType += "_LapseFix"
    else:
        modelType += "_LapseFree"

    if mc_fitter.freeP_c:
        modelType += "_contextualPrior"
    else:
        modelType += "_sharedPrior"

    filename = f"{participantID.split('_')[0]}_{modelType}_simulated.csv"
    filename = os.path.join("simulated_data",participantID.split('_')[0], filename)
    try:
        
        simulatedData= pd.read_csv(filename)
        print(f"Loaded saved simulated data from {filename}")
        return simulatedData

    except:
        user_input = input(f"No saved simulated data found at {filename}. Do you want to simulate and save the data? (yes/no): ").strip().lower()
        if user_input != 'yes':
            print("No simulation found and selected to go with new sim.")
        else:
            return None
        print(f"No saved simulated data found at {filename}")
        fitSaver.saveSimulatedData(mc_fitter, mc_fitter.dataName)
        mc_fitter.simulatedData= pd.read_csv(filename)
        print(f"Simulated data saved to {filename}")
        return mc_fitter.simulatedData
#proceed to simulate and save to {filename}
mc_fitter.simulatedData=loadSimulatedData(mc_fitter, mc_fitter.dataName, participantID=dataName.split(".csv")[0], modelType=mc_fitter.modelName)



Loaded saved simulated data from simulated_data/mt/mt_lognorm_LapseFree_sharedPrior_simulated.csv


In [5]:
# so now we have loaded the simulated data, fitted parameters, loglikelihood, aic, bic

In [ ]:
# Load psychometric fits for both data and simulated data
import importlib
import psychometricFitLoader as pfl_data
import psychometricFitLoader_simulated as pfl_sim

# Get participant ID
dataID = dataName.split("_")[0]

# Load real data psychometric fit
mc_fitter.dataFit = pfl_data.load_psychometric_fit(dataID)
print(f"✅ Loaded real data psychometric fit for {dataID}")
print(f"   AIC: {mc_fitter.dataFit['AIC']:.2f}, Params: {mc_fitter.dataFit['n_params']}")
# Load simulated data psychometric fit
mc_fitter.simulatedFit = pfl_sim.load_psychometric_fit_simulated(dataID, 'lognorm_LapseFree_sharedPrior')
print(f"✅ Loaded simulated data psychometric fit for {dataID}")
print(f"   AIC: {mc_fitter.simulatedFit['AIC']:.2f}, Params: {mc_fitter.simulatedFit['n_params']}")


✅ Loaded real data psychometric fit for mt
   AIC: 2242.25, Params: 42
✅ Loaded simulated data psychometric fit for mt
   AIC: 65648.92, Params: 42


In [ ]:


print(f"✅ Loaded real data psychometric fit for {dataID}")
print(f"   AIC: {mc_fitter.dataFit['AIC']:.2f}, Params: {mc_fitter.dataFit['n_params']}")

# Load simulated data psychometric fit
try:
    mc_fitter.simDataFit = pfl_sim.load_psychometric_fit_simulated(
        dataID, 
        'lognorm_LapseFree_sharedPrior'
    )
    print(f"✅ Loaded simulated psychometric fit for {dataID}")
    print(f"   AIC: {mc_fitter.simDataFit['AIC']:.2f}, Params: {mc_fitter.simDataFit['n_params']}")
    
    # Compare data vs simulated
    comparison = pfl_sim.compare_data_vs_simulated(
        dataID, 
        'lognorm_LapseFree_sharedPrior',
        data_fits_dir='psychometric_fits_data',
        sim_fits_dir='psychometric_fits_simulated'
    )
    print(f"\n📊 Comparison:")
    print(f"   Data AIC: {comparison['data_AIC']:.2f}")
    print(f"   Simulated AIC: {comparison['simulated_AIC']:.2f}")
    print(f"   Delta AIC: {comparison['delta_AIC']:.2f}")
    
except FileNotFoundError as e:
    print(f"⚠️  Simulated fit not found: {e}")
    print(f"   Run: python runDataFitter_simulated.py")

# Plot the fits